In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
DEVICE='cuda'

import cv2
import sys
import numpy as np

import optuna
from pathlib import Path
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import catalyst
from catalyst import dl
from catalyst import metrics

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import math
import pandas as pd
import torch.nn.functional as F

from src import config

In [2]:
from src.architecture import LumbarHmModel
from src.preprocessing import LumbarDataset, get_aug_test, randAugment

In [3]:
data = pd.read_csv(config.PATH_TO_DATA)
train, test = train_test_split(data, test_size=0.2, random_state=777)

In [6]:
model = LumbarHmModel("efficientnet-b2", "Unet")

In [8]:
train_dataset = LumbarDataset(train, tfms=get_aug_test())
dataloader_train = DataLoader(train_dataset, batch_size=16, shuffle=True)

test_dataset = LumbarDataset(test, tfms=get_aug_test())
dataloader_val = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
criterion = nn.MSELoss()
model = HmBaseline().to('cuda:0')
optimizer = torch.optim.AdamW(model.parameters())